In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import backtrader as bt
from datetime import datetime
from sklearn.ensemble import GradientBoostingRegressor
import pandas as pd
import matplotlib.pyplot as plt
import backtrader as bt
from datetime import datetime
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn import linear_model
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [12]:
class firstStrategy(bt.Strategy):
    # example
    def __init__(self):
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=25)
        
    def next(self):
        if not self.position:
            if self.rsi < 30:
                self.buy(size=100)
        else:
            if self.rsi > 70:
                self.sell(size=100)

In [13]:
class SMACross(bt.Strategy):
    # SMA example
    def __init__(self):
        pfast = 5
        pslow = 40
        sma1 = bt.ind.SMA(period=pfast)
        sma2 = bt.ind.SMA(period=pslow)
        self.crossover = bt.ind.CrossOver(sma1, sma2)
    
    def next(self):
        if not self.position:
            if self.crossover > 0:
                self.order_target_percent(target=0.99) # all in
        elif self.crossover < 0:
            self.order_target_size(target=0)

In [14]:
# https://www.backtrader.com/blog/2019-06-13-buy-and-hold/buy-and-hold/
class SimpleBuyAndHold(bt.Strategy):
    def start(self):
        self.val_start = self.broker.get_cash()  # keep the starting cash
        self.size = 0
        self.days = 0

    def nextstart(self):
        self.order_target_percent(target=0.99)
        print(self.data.datetime.time())
    
    def next(self):
        if self.days == 365:
            self.close()
        self.days += 1

    def stop(self):
        print("Position:", self.getposition().size)
        self.roi = (self.broker.get_value() / self.val_start) - 1.0
        print('ROI:        {:.2f}%'.format(100.0 * self.roi))        

In [15]:
class oneYear(bt.Strategy):
    def start(self):
        self.val_start = self.broker.get_cash()  #
        self.size = 0
        self.days = 0

    def nextstart(self):
        self.order_target_percent(target=0.99) 
        print(self.data.datetime.time())
    
    def next(self):
        if self.days == 365:
            self.close()
        self.days += 1

    def stop(self):
        # calculate the actual returns
        print("Position:", self.getposition().size)
        self.roi = (self.broker.get_value() / self.val_start) - 1.0
        print('ROI:        {:.2f}%'.format(100.0 * self.roi))        

In [16]:

class PolyStrat(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.val_start = self.broker.get_cash()
        self.dataclose = self.datas[0].close
        self.buy_count = 0
        self.sell_count = 0
    '''
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('Buy executed {}'.fromat(order.executed.price))
                
            if order.issell():
                self.log('Sell executed {}'.fromat(order.executed.price))
                
            self.bar_executed = len(self)
        self.order = None;
    '''    
    def next(self):
        # Simply log the closing price of the series from the reference
#         self.log('Close, %.2f' % self.dataclose[0])
        
        days = 10
        data = {
                'Close' : [self.dataclose[i] for i in range(-days, 0)][::-1],
                'row_num' : [i for i in range(1, days + 1)]
            }
        df = pd.DataFrame (data, columns = ['Close', 'row_num'])
        stock_df = df
        
        X = stock_df[['row_num']]
        y = stock_df.Close
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
        #poly_features = PolynomialFeatures(degree = 2, include_bias = False)
        #X_poly = poly_features.fit_transform(X_train)
        #reg_f = LinearRegression()
        #reg_f.fit(X_poly, y_train)
        
        gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=36)
        X = stock_df[['row_num']]
        y = stock_df.Close
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
        gbrt.fit(X_train, y_train)
        
        
        predicted = gbrt.predict([[days+1]])[0]
        
        if self.dataclose[0] < predicted:
            if (len(self.position) == 0):
                self.order = self.order_target_percent(target=1)
                self.buy_count += 1
        if self.dataclose[0] > predicted:
            self.order = self.sell(size=len(self.position))
            self.sell_count += 1
#         print(len(self.position))
                
                
                
                
                
#         #             self.log('BUY CREATE, %.2f' %self.dataclose[0] )
# #                 self.order = self.order_target_percent()
#                 self.buy(size=1)
#                 self.buy_count += 1
#         if self.dataclose[0] > predicted:
#             if self.position:
#                 for data in self.datas:
#                     size = self.getposition(data).size
#                     if size != 0:
#                         self.close(data)
#                         self.sell_count += 1

                
#                 self.order = self.close()
    '''
    
        if not self.position:
            data = {
                'Close' : [self.dataclose[-1], self.dataclose[-2], self.dataclose[-3], self.dataclose[-4], self.dataclose[-5],
                            self.dataclose[-6], self.dataclose[-7], self.dataclose[-8], self.dataclose[-9], self.dataclose[-10],
                            self.dataclose[-11], self.dataclose[-12], self.dataclose[-13],self.dataclose[-14],
                            self.dataclose[-15],self.dataclose[-16],self.dataclose[-17],self.dataclose[-18],
                             self.dataclose[-19],self.dataclose[-20]],
                'row_num' : [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
            }
            df = pd.DataFrame (data, columns = ['Close', 'row_num'])
            stock_df = df
            X = stock_df[['row_num']]
            y = stock_df.Close
            X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
            poly_features = PolynomialFeatures(degree = 2, include_bias = False)
            X_poly = poly_features.fit_transform(X_train)
            reg_f = LinearRegression()
            reg_f.fit(X_poly, y_train)
            predicted = reg_f.predict([[21, 441]])[0]
            if self.dataclose[0] < predicted:
                self.log('BUY CREATE, %.2f' %self.dataclose[0] )
                self.order = self.buy()
        else:
            if len(self) >= (self.bar_executed + 5)
                self.lof('Sell, %.2f' %self.dataclose[0])
                self.order = self.sell()
        '''
#TODO: Sell stock and buy appropriate quantity
        
    
    
    def stop(self):
        # calculate the actual returns
        print("Position:", self.getposition().size)
        self.roi = (self.broker.get_value() / self.val_start) - 1.0
        print('ROI:        {:.2f}%'.format(100.0 * self.roi))
        print('Buy count:', self.buy_count)
        print('Sell count:', self.sell_count)

In [17]:
# from https://backtest-rookies.com/2017/06/02/getting-started-first-script/

# set starting cash
startcash = 1000000

cerebro = bt.Cerebro()
broker = bt.brokers.BackBroker(shortcash=False)
cerebro.broker = broker 

# COPY PASTE strategy name here
# cerebro.addstrategy(STRATEGYNAME)
cerebro.addstrategy(PolyStrat)


# Get SPY data from Yahoo Finance
data = bt.feeds.YahooFinanceData(dataname='SPY',
                                 fromdate = datetime(2017, 1, 1),
                                 todate = datetime(2020, 9, 30)
                                )
cerebro.adddata(data)
cerebro.broker.setcash(startcash)

print('Starting Portfolio Value: $%.2f' % cerebro.broker.getvalue())

# start backtest here
cerebro.run()

#Get final portfolio Value
portvalue = cerebro.broker.getvalue()
pnl = portvalue - startcash

#Print out the final result
print('Final Portfolio Value: $%.2f' % cerebro.broker.getvalue())
print('P/L: ${}'.format(pnl))

#Finally plot the end results
cerebro.plot()

Starting Portfolio Value: $1000000.00
Position: 3139
ROI:        5.98%
Buy count: 77
Sell count: 649
Final Portfolio Value: $1059819.48
P/L: $59819.480000001146


ImportError: cannot import name 'warnings' from 'matplotlib.dates' (/home/aryaman/.local/lib/python3.8/site-packages/matplotlib/dates.py)

In [79]:
# buy and hold P/L: $576803.3700000001